In [42]:
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve &>/dev/null &
print("Ollama server started in the background.")

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Ollama server started in the background.


In [43]:
from IPython import get_ipython
from IPython.display import display
import time # Import the time module
import sys
import requests

In [44]:
# Optional: Check if Ollama server is running
max_retries = 60  # Increase the number of retries
retry_delay = 5   # seconds to wait between retries
initial_delay = 10 # Add an initial delay before the first attempt

time.sleep(initial_delay) # Wait for initial_delay seconds before the first attempt

for attempt in range(max_retries):
    try:
        # Add a timeout to the request, can also increase if needed
        response = requests.get("http://localhost:11434/", timeout=10)
        if response.status_code == 200:
            print(f"Ollama server is reachable after {attempt + 1} attempts.")
            break # Exit the loop if connection is successful
        else:
            print(f"Attempt {attempt + 1}: Ollama server returned status code: {response.status_code}")
    except requests.exceptions.ConnectionError as e:
        print(f"Attempt {attempt + 1}: Could not reach Ollama server: {e}")

    if attempt < max_retries - 1:
        time.sleep(retry_delay)
    else:
        # Exit if max retries reached, print final error message
        sys.exit("Ollama server not running after multiple attempts. Please check the installation and execution.")

Ollama server is reachable after 1 attempts.


In [45]:
!ollama pull llama3

In [46]:
pip install langchain langchain-community chromadb fastembed pypdf


In [47]:
!pip install langchain_community

In [48]:
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import sys

In [60]:
def ingest():
    # Get the doc
    loader = PyPDFLoader("/content/Travel management application revised.pdf")
    pages = loader.load_and_split()
    # Split the pages by char
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1024,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(pages)
    print(f"Split {len(pages)} documents into {len(chunks)} chunks.")
    #
    embedding = FastEmbedEmbeddings()
    #Create vector store
    Chroma.from_documents(documents=chunks,  embedding=embedding, persist_directory="./sql_chroma_db")


In [61]:
ingest()

Split 21 documents into 37 chunks.


In [ ]:
import os
token = os.getenv("HUGGINGFACE_TOKEN")
from huggingface_hub import login
access_token_read = "token"
access_token_write = "token"
login(token = access_token_read)

In [63]:
def rag_chain(pdf_path="/content/Travel management application revised.pdf", persist_dir="./my_pdf_chroma_db"):
    # Load PDF
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()

    # Split text
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(pages)

    # Embed and persist
    embedding = FastEmbedEmbeddings()
    vector_store = Chroma.from_documents(
        documents=docs,
        embedding=embedding,
        persist_directory=persist_dir,
    )
    vector_store.persist()

    # Prompt template
    prompt = PromptTemplate.from_template(
        """
        <s> [Instructions] You are a friendly assistant. Answer the question based only on the following context.
        If you don't know the answer, then reply, No Context available for this question {input}. [/Instructions] </s>
        [Instructions] Question: {input}
        Context: {context}
        Answer: [/Instructions]
        """
    )

    # Set up retriever and chain
    retriever = vector_store.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={"k": 3, "score_threshold": 0.5},
    )

    model = ChatOllama(model="llama3")
    document_chain = create_stuff_documents_chain(model, prompt)
    chain = create_retrieval_chain(retriever, document_chain)

    return chain

In [64]:
def ask(query: str):
    #
    chain = rag_chain()
    # invoke chain
    result = chain.invoke({"input": query})
    # print results
    print(result["answer"])
    for doc in result["context"]:
        print("Source: ", doc.metadata["source"])

In [66]:
ask("print third page of uploaded pdf")

You want to print the third page of an uploaded PDF!

Answer: Yes! I can help you with that. Since you've uploaded a PDF, I'll assume it's a digital file. To print the third page, you can follow these steps:

1. Open the uploaded PDF in your preferred PDF viewer or editor.
2. Navigate to the third page by using the page navigation controls (e.g., page up/down buttons or keyboard shortcuts like Ctrl+Page Down/Page Up).
3. Once on the third page, click the "Print" button or press Ctrl+P (Windows) or Command+P (Mac).
4. In the print dialog box, select your desired printer and any additional printing settings as needed.
5. Click "Print" to send the third page to your selected printer.

If you need further assistance or have any questions about these steps, feel free to ask!


In [70]:
ask("what is travel management application")

A Travel Management Application is a platform that simplifies travel planning by integrating multiple services into one place. It aims to provide efficient performance, user-friendliness, and secure data handling using technologies such as C++, Qt, and SQL. This application demonstrates how modern technologies can revolutionize the tourism industry, making travel more convenient and enjoyable for users.
Source:  /content/Travel management application revised.pdf
Source:  /content/Travel management application revised.pdf
Source:  /content/Travel management application revised.pdf


In [71]:
ask("what are the methodology used in this application")

Based on the provided context, the methodologies used in this application are:

1. Git & GitHub for version control and collaboration.
2. Functions and Conditional Statements for developing various aspects of the game.

No other methodologies are mentioned in the given context.
Source:  /content/Travel management application revised.pdf
Source:  /content/Travel management application revised.pdf
Source:  /content/Travel management application revised.pdf
